In [ ]:
#User Input
#lib_type = "single_end"
#seq_platform = "illumina"
#read_length = 100

In [ ]:
source("https://bioconductor.org/biocLite.R")
#biocLite("dada2")
#install.packages("phyloseq")
#biocLite("biom")
biocLite("themetagenomics")

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.4 (2018-03-15).
Installing package(s) ‘themetagenomics’
also installing the dependencies ‘coda’, ‘bindr’, ‘extrafontdb’, ‘Rttf2pt1’, ‘statnet.common’, ‘reticulate’, ‘ISOcodes’, ‘yaml’, ‘tidyselect’, ‘bindrcpp’, ‘plogr’, ‘extrafont’, ‘fastmatch’, ‘ggrepel’, ‘lubridate’, ‘network’, ‘RSpectra’, ‘sna’, ‘SnowballC’, ‘spacyr’, ‘stopwords’, ‘xml2’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘htmlwidgets’, ‘tidyr’, ‘hexbin’, ‘dplyr’, ‘crosstalk’, ‘purrr’, ‘StanHeaders’, ‘inline’, ‘gridExtra’, ‘slam’, ‘quanteda’, ‘glmnet’, ‘quadprog’, ‘RcppArmadillo’, ‘lda’, ‘lme4’, ‘plotly’, ‘rstan’, ‘stm’



In [ ]:
library("dada2")
library("phyloseq")
library("biom")
library("themetagenomics")

In [ ]:
#Read in files

input_files = c("../ERR520895.fastq", "../ERR521461.fastq")
file_names = c("ERR520895", "ERR521461")

In [ ]:
#Plot quality profiles of files

plotQualityProfile(input_files)

In [ ]:
#Filter and Trim the reads

filtFs = paste0(file_names,"_filt.fastq")
filterAndTrim(input_files, filtFs, compress=TRUE, verbose=TRUE, multithread=TRUE)
plotQualityProfile(filtFs)

In [ ]:
#Parse filtered files to learn error rates, infer sequence variants and construct a seq-table

#filtered_files = list.files(filtpath, pattern="fastq.gz", full.names=TRUE)
#samples = sapply(basename(filtFs))
#names(filtFs) = basename(input_files)

error_rates = learnErrors(filtFs, multithread=TRUE, randomize=TRUE)
plotErrors(error_rates, nominalQ=TRUE)


In [ ]:
seq_variants = vector("list", length(file_names))
names(seq_variants) = filtFs

for (i in filtFs){
    #dereplicated = derepFastq(filtFs[[i]])
    dereplicated = derepFastq(i)
    seq_variants[[i]] = dada(dereplicated, err=error_rates, multithread=TRUE)}
    
sequence_table = makeSequenceTable(seq_variants)

In [ ]:
#Remove chimeras and assign taxonomy
system("wget https://zenodo.org/record/158955/files/gg_13_8_train_set_97.fa.gz?download=1")
sequence_table = removeBimeraDenovo(sequence_table, method="consensus", multithread=TRUE)
taxa_assign = assignTaxonomy(sequence_table, "gg_13_8_train_set_97.fa.gz", multithread=TRUE)



In [ ]:
#Create Metadata and Get OTU Table

otu = otu_table(sequence_table, taxa_are_rows=FALSE)


In [ ]:
#Get pathway annotation using picrust

tmp = tempdir()
download_ref(tmp,reference='gg_ko',overwrite=FALSE)
system.time(FUNCTIONS <- picrust(otu,rows_are_taxa=FALSE,
                                 reference='gg_ko',reference_path=tmp,
                                 cn_normalize=TRUE,sample_normalize=FALSE,
                                 drop=TRUE))